Практическое задание к уроку № 10

In [10]:
import cv2
import glob
from google.protobuf.json_format import MessageToDict
import mediapipe as mp
import numpy as np
import pandas as pd
import re
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [11]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_detection = mp.solutions.face_detection
mp_hands = mp.solutions.hands

В качестве датасета для распознавания жестов используем один из  
датасетов kaggle. (https://www.kaggle.com/datasets/gti-upm/leapgestrecog)  
В этом датасете 10 разных людей показывают 10 жестов. Из этих 10 жестов  
мы выберем 3, на которые будет реагировать модель. Также возьмём еще 3 жеста,  
которые будут принадлежать к классу разное, чтобы модель понимала, что кроме  
нужных нам жестов, пользователь может показывать и другие, а не причисляла  
любой жест к одному из трёх целевых.  
  
Получим необходимые имена файлов, относящиеся к 6 классам:  

In [12]:
files = glob.glob('./leapGestRecog/*/0[13457]*/*') + glob.glob('./leapGestRecog/*/10*/*')


Напишем функцию, которая на вход будет принимать особые точки жеста,  
а возвращать их масштабированные координаты. При формировании обучающего  
датафрейма будем ещё добавлять таргет:

In [13]:
def landmarks_to_df(landmarks, df=None, target=None):
    
    # Переводим в привычный словарь
    landmarks = MessageToDict(landmarks[0])['landmark']
    
    # Заполняем списки с координатами
    x = np.empty(len(landmarks))
    y = np.empty(len(landmarks))
    z = np.empty(len(landmarks))
    
    for i, v in enumerate(landmarks):
        x[i] = v['x']
        y[i] = v['y']
        z[i] = v['z']
    
    # Масшабируем координаты, чтобы не имело значения, в какой
    # области кадра наш жест
    scaler = MinMaxScaler()
    x = scaler.fit_transform(x.reshape(-1, 1)).reshape(1, -1)
    y = scaler.fit_transform(y.reshape(-1, 1)).reshape(1, -1)
    z = scaler.fit_transform(z.reshape(-1, 1)).reshape(1, -1)
    
    # Подготавливаем и возвращаем датасет
    if target is not None:
        features = np.c_[x, y, z, target]
    else:
        features = np.c_[x, y, z]
    
    if df is not None:
        df = pd.concat((df, pd.DataFrame(features)), axis=0)
    else:
        df = pd.DataFrame(features)
    
    return df

Чтобы внести координаты особых точек в датафрейм, сначала нужно найти  
эти точки. Для этого прогоняем все обучающие картинки через фреймворк,  
а затем вносим их обработанные координаты в датафрейм. Также присваиваем  
значения таргета жестам, где 3 целевых жеста получают свой класс, а  
остальные 3 жеста получают нулевой класс. Картинки, где фреймфорк не  
обнаружил жест, пропускаются.  

In [14]:
mapping_dict = {'03': 1, '07': 2, '10': 3}
df = pd.DataFrame()

with mp_hands.Hands(max_num_hands=1, static_image_mode=True) as hands:
    
        for idx, file in enumerate(files):
            image = cv2.imread(file)
            results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            if not results.multi_hand_landmarks:
                continue
            
            target = mapping_dict.get(re.findall('frame_\d{2}_(\d{2})', file)[0], 0)
            df = landmarks_to_df(results.multi_hand_landmarks, df, target)

Посмотрим на получившийся датафрейм:

In [15]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.026055,0.000000,0.107609,0.247835,0.398601,0.489592,0.618835,0.509207,0.413254,0.558857,...,0.000000,0.692860,0.264774,0.064647,0.042940,0.478142,0.244879,0.138012,0.101530,2.0
0,0.000000,0.322499,0.609520,0.829921,1.000000,0.263444,0.520810,0.735434,0.914113,0.192786,...,0.008541,0.967427,0.666530,0.276842,0.031134,0.887575,0.585671,0.330568,0.158112,2.0
0,0.000000,0.000762,0.161427,0.402390,0.614149,0.080991,0.258220,0.422858,0.569542,0.243840,...,0.087132,1.000000,0.606149,0.226274,0.000000,0.989427,0.608881,0.352394,0.191209,2.0
0,0.010466,0.000000,0.160569,0.409021,0.619338,0.147174,0.282111,0.409574,0.527152,0.296486,...,0.207363,0.823889,0.528559,0.202127,0.000000,0.647173,0.412920,0.266200,0.168350,2.0
0,0.000000,0.196373,0.438858,0.714953,0.957425,0.168564,0.440325,0.673140,0.862912,0.250065,...,0.229337,0.796692,0.632724,0.265196,0.003328,0.616691,0.423486,0.210361,0.045881,2.0


In [16]:
df.shape

(5878, 64)

In [17]:
df[63].value_counts()

63
0.0    2579
2.0    1610
3.0    1169
1.0     520
Name: count, dtype: int64

Баланс классов удовлетворительный, сохраним датасет:



In [18]:
df.to_csv('./data.csv', index=False)

Ячейка для загрузки датасета из файла:

In [19]:
df = pd.read_csv('./data.csv')
df.columns = df.columns.astype(int)

Разделим данные на обучающую и тестовую выборки, сделаем  
стратификацию по таргету:

In [20]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1], test_size=0.2, stratify=df[63], random_state=29)


Обучим логистическую регрессию:

In [21]:

model = LogisticRegression(max_iter=400)
model.fit(X_train, y_train)
preds = model.predict(X_test)

Посмотрим на метрики:

In [22]:
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99       516
         1.0       0.99      0.98      0.99       104
         2.0       0.98      0.99      0.99       322
         3.0       0.99      0.95      0.97       234

    accuracy                           0.99      1176
   macro avg       0.99      0.98      0.98      1176
weighted avg       0.99      0.99      0.99      1176



Модель очень хорошо научилась разделять классы.  
Перед использованием заново обучим её на всём датасете:

In [23]:
model.fit(df.iloc[:, :-1], df.iloc[:, -1])


LogisticRegression(max_iter=400)

Теперь у нас есть всё для работы программы. Будем брать изображение  
с веб-камеры и детектировать на нём лицо. Если лицо не обнаружено, то  
выводится соответствующее сообщение. Если обнаружено, то производится  
поиск жеста. При обнаружении жеста, координаты особых точек передаются  
модели классификации, и, в зависимости от жеста, на экран выводится  
соответствующее сообщение

In [ ]:
cap = cv2.VideoCapture(0)

with mp_face_detection.FaceDetection() as face_detection:
    with mp_hands.Hands(max_num_hands=1) as hands:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print('Ignoring empty camera frame.')
                continue

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = face_detection.process(image)
            
            # Несмотря на наличие параметра min_detection_confidence у FaceDetection(), он,  
            # похоже, не работает. Поэтому зададим порог уверенности вручную (0,85), чтобы  
            # избежать ложных срабатываний детектирования лица
            if results.detections and MessageToDict(results.detections[0])['score'][0] > 0.85:
                
                results = hands.process(image)
                
                if results.multi_hand_landmarks:
                    gesture = model.predict_proba(landmarks_to_df(results.multi_hand_landmarks))[0]
                    
                    if max(gesture) > 0.5:
                        gesture = np.argmax(gesture)
                        
                        match gesture:
                            case 0:
                                text = 'Unknown sign!'
                            case 1:
                                text = 'Hello!'
                            case 2:
                                text = 'OK!'
                            case 3:
                                text = 'Scary!'
                    
                    # Если модель не уверена во всех классах, но фреймворк всё равно находит жест,
                    # на экран будет выводиться сообщение
                    else:
                        text = 'Is it a sign?'
            
                
                    cv2.putText(image, text, (170, 50),
                                cv2.FONT_HERSHEY_COMPLEX,
                                1.3, (0, 0, 255), 2)
            
            else:
                cv2.putText(image, 'Is anyone here?', (170, 50),
                            cv2.FONT_HERSHEY_COMPLEX,
                            1.3, (0, 0, 255), 2)
            
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            cv2.imshow('MediaPipe Hands', image)
            if cv2.waitKey(5) & 0xFF == 27:
                break

cap.release()
cv2.destroyAllWindows()